In [21]:
defi_template = r'C:\Users\jupadhyay\Desktop\Medicaid\24281193\Deficiencies Template_NEW.xlsx'
data_location = r"C://Users//jupadhyay//Desktop//def//"
RequestedBy = '1059'
Requestorname = 'John Akindude'
RequestedDate = '4/08/2022'
Ticket = '24281193'
username = 'jupadhyay' 
password = 'Jayrang$456'

## Deficiency Request on and Request by File_PS 

In [22]:
import os
import pandas as pd
import glob
import xlwings as xw
import pyodbc 

server = 'reportdb01.archer.local' 
database = 'S3Reporting' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
clientnote = (
'Deficiency requested on '+ RequestedDate+' and requested by '+Requestorname+', updated deficiency RequestedBy and '
'RequestedDate as per instructions given by '+Requestorname+' on ticket #'+Ticket
)
heading = ["Client Id", "Deficiency Id"]
df_total = pd.DataFrame(columns = heading)

for file in os.listdir(data_location) :
    df_file = pd.read_excel(data_location + file, sheet_name = 0)
    df2= pd.DataFrame(df_file)
    selected_columns = df2.loc[:, heading]
    df_total = pd.concat([selected_columns, df_total], ignore_index = True)


DefiId = df_total["Deficiency Id"].astype(str).str.cat(sep=",")

defiquery = (
'select DeficiencyId, EntityId, EntityTypeId, EntityTypeName, DeficiencyTypeId, TypeName, StageId, StageName, Note, CreatedBy, CreatedByName, RequestedBy, RequestedDate, CuredBy, CuredDate, FollowUpDate \n'
'from Deficiencies \n'
'where DeficiencyId in ('+DefiId+')'
)
defidf = pd.read_sql_query(defiquery, cnxn)
joindf = pd.merge(df_total, defidf, how='inner', left_on='Deficiency Id', right_on='DeficiencyId')
defiupdate = joindf = joindf.query('StageId !=4')

clientupdate = pd.DataFrame(defiupdate['Client Id'].drop_duplicates())
clientupdate.rename(columns={'Client Id':'Id'}, inplace = True)
clientupdate['NewClientNote']=clientnote

defiupdate = pd.DataFrame(defiupdate[['DeficiencyId','StageId','RequestedBy','RequestedDate']].drop_duplicates())
defiupdate.loc[:,'StageId'] = '3'
defiupdate.loc[:,'RequestedBy'] = RequestedBy
defiupdate.loc[:,'RequestedDate'] = RequestedDate

wb = xw.Book(defi_template)
original = wb.sheets['Original']
wip = wb.sheets['WIP']
defi_update = wb.sheets['Deficiency Update']
client_update = wb.sheets['Client Update']
defisql_pre = wb.sheets['Deficiency SQL (pre)']
defisql_post = wb.sheets['Deficiency SQL (post)']

original.range('A1').options(pd.DataFrame, index=False).value = df_total
wip.range('A1').options(pd.DataFrame, index=False).value = df_total
defi_update.range('A1').options(pd.DataFrame, index=False).value = defiupdate
client_update.range('A1').options(pd.DataFrame, index=False).value = clientupdate
defisql_pre.range('A1').options(pd.DataFrame, index=False).value = defidf
wb.save()

